# Homework 3 Code For CVXPY Additional Problems

In [16]:
import cvxpy as cp
import numpy as np

## Problem 1 - Exercise 4.4

In [57]:
#Finding the optimal activity level for exercise 4.17

#Defining the problem's parameters 
A = np.array([[1,2,0,1], [0,0,3,1], [0,3,1,1], [2,1,2,5], [1,0,3,2]])
c_max = np.array([100,100,100,100,100])
p = np.array([3,2,7,6])
p_disc = np.array([2,1,4,2])
q = np.array([4,10,5,10]) 

#Decision variables
x_under = cp.Variable(4)
x_over = cp.Variable(4)

#Opt problem
obj = cp.Minimize(-(p.T @ x_under + p_disc.T @ x_over))
constraints = [
  x_under >= 0, 
  x_under <= q,
  x_over >= 0, 
  x_under -q <= x_over, #Equal was a bad constraint
  A @ (x_under + x_over) <= c_max
]

prob = cp.Problem(obj, constraints)


f_opt = -prob.solve()
x_opt = x_over.value + x_under.value
rev_per_activity = cp.multiply(p.T,x_under.value).value + cp.multiply(p_disc.T, x_over.value).value
avg_rev_per_activity = rev_per_activity / x_opt

print(f"Optimal Activity levels: {x_opt}")
print(f"Optimal revenue: {f_opt}")
print(f"Revenue per activity: {rev_per_activity}")
print(f"Average Revenue per activity: {avg_rev_per_activity}")
np.round(x_under.value,2), np.round(x_over.value,2)


Optimal Activity levels: [ 4.00000003 22.49999991 30.99999993  1.50000004]
Optimal revenue: 192.4999998042132
Revenue per activity: [ 11.99999996  32.49999988 138.99999975   9.00000022]
Average Revenue per activity: [2.99999997 1.44444445 4.48387097 6.        ]


(array([ 4. , 10. ,  5. ,  1.5]), array([ 0. , 12.5, 26. , -0. ]))

Here the optimal activity levels are [4, 22.5, 31, 1.5] and it delivers a total revenue of 192.5. Checking if this solution is valid, we see that the constraints are satisfied as the optimal values are greater than zero and add up to incorporate the discount quantity. The objective function value is also correct as it is the sum of the revenue from each product.
* 4 x 3 = 12
* 10 x 2 + 12.5 x 1 = 32.5
* 5 x 7 + 26 x 4 = 35 + 104 = 139
* 1.5 x 6 = 9
* Total = 12 + 32.5 + 139 + 9 = 44.5 + 148 = 192.5

## Problem 2 - No longer in book but similar to 4.3

In [83]:
#Set up
x = cp.Variable(1)
y = cp.Variable(1)
z = cp.Variable(1)
c = np.arange(1)

obj = cp.Minimize(c @ x)

####  Part a

The constraint $\| x+2y, x-y \| = 0$ is the same as $\sqrt{(x+2y)^2 + (x-y)^2} = 0$. This is only valid when $x+2y$ and $x-y$ both equal zero. If you solve that system of equations it gives you $x=0, y=0$. The original constraint is invalid because it is an equality constraint that is not affine.

In [84]:
constraints = [x== 0, y==0]
prob = cp.Problem(obj, constraints)

f_opt = prob.solve()
f_opt

0.0

####  Part b

The constraint $((x+y)^2)^2 \leq x-y$ is invalid because the arguments of a square must be affine to ensure they preserve convexity. Distributing it to $(x+y)^4  - x + y\leq 0$ makes it valid.

In [85]:
constraints = [(x+y)**4 - x + y <= 0]
prob = cp.Problem(obj, constraints)

f_opt = prob.solve()
f_opt

0.0

####  Part c

This one is just using the inv_pos function because it excludes zero from the domain.

In [86]:
constraints = [cp.inv_pos(x) + cp.inv_pos(y) <= 1, x >= 0, y >= 0]
prob = cp.Problem(obj, constraints)

f_opt = prob.solve()
f_opt

0.0

####  Part d

The constraint norm([ max( x , 1 ) , max( y , 2 ) ]) <= 3x + y is equal to $\sqrt{max(x,1)^2 + max(y,2)^2} \leq 3x + y$. The problem here is that the norm function needs to take in an affine argument. So we can make variables that are equal to the maximum of x and 1, and y and 2. Then we can use those variables in the norm function.

In [114]:
u = cp.maximum(cp.vstack([x,1]),1)
v = cp.maximum(cp.vstack([y,2],),1)
constraints = [cp.norm(cp.vstack([u,v])) <= 3*x + y]
prob = cp.Problem(obj, constraints)

f_opt = prob.solve()
f_opt

0.0

####  Part e

We can rearrange the problem so $\frac{1}{y} - x \leq 0$ and $x,y \geq 0$.

In [116]:
constraints = [cp.inv_pos(y) - x <= 0, x>= 0, y>= 0]
prob = cp.Problem(obj, constraints)

f_opt = prob.solve()
f_opt

0.0

####  Part f

The constraint $\frac{(x+y)^2}{\sqrt{y}} \leq x-y +5$ is convex because of the composition rules. The right hand side is affine and can be brought over

In [120]:
constraints = [ cp.quad_over_lin((x+y)**2,cp.sqrt(y)) - x + y - 5 <=0]
prob = cp.Problem(obj, constraints)

f_opt = prob.solve()
f_opt

0.0

####  Part g

The constraint $x^3 + y^3<= 1, x>= 0 ,y >= 0$ fails because the left hand side is not convex. However, iti s on the interval $x>=0$ so we can use the power function

In [121]:
constraints = [cp.power(x,3) + cp.power(y,3) <= 1]
prob = cp.Problem(obj, constraints)

f_opt = prob.solve()
f_opt

0.0

####  Part h

The constraint $x+z \leq 1 + \sqrt{x*y -z^2}, x\geq 0, y \geq 0 $ is not convex because of the square root. We can rearrange it to be $\frac{(x+z-1)^2 +z^2}{y} - x \leq 0$. This is a convex function since the numerator is convex, and the denominator is affine.  It can also be seen as the quad over lin for two terms and then affine in all three.

In [122]:
constraints = [cp.quad_over_lin(x+z-1,y) + cp.quad_over_lin(z,y) -x <= 0, x>= 0, y>= 0]
prob = cp.Problem(obj, constraints)

f_opt = prob.solve()
f_opt

0.0

## Problem 3. Exercise 4.7

#### Part a

$e^{max|\log a_i^T p - \log I_{des} |}$. This is a convex because $I_{des}$ is a constant the minimum of concave functions is concave. The exponential function is convex and using composition rules, since the exponential is convex and non-increasing, and the inside function is concave, the function is convex.